# Fine-Tuning d'un Modèle

## Introduction

Dans ce notebook, nous allons effectuer le fine-tuning d'un modèle à partir de l'API Transformers d'Hugging Face. Les étapes suivantes seront suivies :

1. Importation des bibliothèques nécessaires.
2. Récupération des données.
3. Prétraitement des données.
4. Récupération du modèle.
5. Entraînement du modèle à partir des données.

## Importation des bibliothèques nécessaires

In [1]:
import numpy as np
import evaluate
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
import torch
from tqdm.auto import tqdm

## Récupération des données

Pour récupérer un jeu de données sur le Hub, il suffit d'utiliser la fonction `load_dataset` :

In [2]:
dataset = load_dataset("yelp_review_full")

Il s'agit du jeu de données **Yelp Reviews**. Chaque donnée correspond à un avis et une note (1, 2, 3, 4 ou 5)

**Exemple (d'une donnée)** :
- Avis : Can't miss stop for the best Fish Sandwich in Pittsburgh.
- Note : 5 étoiles

Le jeu de données est séparé en deux parties : les données d'**entraînement** et les données de **test**.

Ces deux parties sont regroupées dans une sorte de dictionnaire, nommé `DatasetDict` :

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

Pour accéder à la partie entraînement (respectivement test), on doit mettre entre crochets `"train"` (respectivement `"test"`) :

In [4]:
dataset["train"]

Dataset({
    features: ['label', 'text'],
    num_rows: 650000
})

On obtient ainsi un objet de type `Dataset`.

Pour accéder à une donnée d'un objet de type `Dataset`, il suffit de mettre entre crochets l'indice de la donnée dans le jeu de données :

In [5]:
dataset["train"][15]

{'label': 4,
 'text': "Can't miss stop for the best Fish Sandwich in Pittsburgh."}

## Prétraitement des données

À présent, on doit prétraiter les données pour qu'on puisse par la suite les "donner à manger" au modèle.

Pour cela, dans notre exemple, on va récupérer le tokenizer pré-entraîné BERT base :

In [3]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

On définit ensuite une fonction qui permettra de "tokénizer" un "batch" de données à partir du tokenizer récupéré :

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

On peut "tokénizer" les données du jeu de données avec la méthode `map` de la classe `DatasetDict` ou `Dataset` :

In [5]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

Voici un exemple d'une donnée "tokénizée" :

In [20]:
#tokenized_dataset["train"][15]

Il s'agit d'un dictionnaire avec plusieurs caractéristiques :

- `label` : le label, ici la note
- `text` : le texte avant la "tokénization", ici l'avis
- `input_ids` : les indices de chaque token
- `token_type_ids` : TODO
- `attention_mask` : TODO

Voici à quoi ressemble la séquence de tokens d'une donnée "tokénizée" :

In [21]:
#tokenizer.tokenize(tokenizer.decode(tokenized_dataset["train"][15]["input_ids"]))

On peut créer un sous jeu de données du jeu de données initial pour le "fine-tuning" afin de réduire le temps qu'il prend :

In [6]:
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(100))
small_test_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(100))

## Récupération du modèle

Maintenant qu'on s'est occupé des données, on doit récupérer le modèle.

Dans notre exemple, la tâche souhaitée est de prédire une note à partir d'un avis. Il s'agit donc d'un apprentissage supervisé (les données d'entraînement sont munis d'une etiquette, ici la note). On peut voir cette tâche comme une classification de textes. On va ainsi récupérer le modèle de classification de textes pré-entraîné BERT base :

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5) # num_labels indique le nombre de labels possibles (i.e. le nombre de classes)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


*Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.*

## Entraînement du modèle à partir des données

C'est parti ! On peut "fine-tune" le modèle à partir des données !

Il y a deux moyens de le faire :

- Avec l'API Trainer
- Sans l'API Trainer, de manière native avec PyTorch

À présent, vous devez choisir un moyen parmi les deux !

### API Trainer

#### Réglage des hyperparamètres

Il est possible de régler les hyperparamètres :

In [8]:
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch") # eval_strategy="epoch" permet d'afficher l'évaluation de la métrique à la fin de chaque époque

#### Évaluation à partir d'une métrique

On a aussi besoin de posséder une métrique permettant de calculer la performance du modèle pendant l'entraînement, ici une simple fonction de précision (entre la prédiction et le label):

In [9]:
metric = evaluate.load("accuracy")

On peut alors définir une fonction qui va calculer la précision entre la prédiction et le label :

In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

#### Entraînement

On crée un objet de type `Trainer` à partir du modèle, du réglage des hyperparamètres, des données d'entraînement et de test, et de la fonction d'évaluation :

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_test_dataset,
    compute_metrics=compute_metrics,
)

Cet objet permet d'effectuer l'entraînement :

In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.613454,0.190000
2,No log,1.591599,0.280000
3,No log,1.584950,0.280000


TrainOutput(global_step=39, training_loss=1.5853855426494892, metrics={'train_runtime': 53.6868, 'train_samples_per_second': 5.588, 'train_steps_per_second': 0.726, 'total_flos': 78935442739200.0, 'train_loss': 1.5853855426494892, 'epoch': 3.0})

### Native avec PyTorch

On doit supprimer la colonne "text" et renommer la colonne "label" en "labels" :

In [10]:
small_train_dataset = small_train_dataset.remove_columns(["text"])
small_test_dataset = small_test_dataset.remove_columns(["text"])

small_train_dataset = small_train_dataset.rename_column("label", "labels")
small_test_dataset = small_test_dataset.rename_column("label", "labels")

On modifie le format pour obtenir des tenseurs PyTorch :

In [12]:
small_train_dataset.set_format("torch")
small_test_dataset.set_format("torch")

On crée un objet de type `DataLoader` pour les données d'entraînement et de test afin de pouvoir itérer sur les "batches" de données :

In [13]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
test_dataloader = DataLoader(small_test_dataset, batch_size=8)

On récupère un optimiseur :

In [14]:
optimizer = AdamW(model.parameters(), lr=5e-5)

On récupère "the default learning rate scheduler" (à partir de `Trainer`):

In [15]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

On essaie si possible d'utiliser comme "device" un GPU (pour aller plus vite) :

In [16]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

C'est parti ! On entraîne le modèle :

In [17]:
progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/39 [00:00<?, ?it/s]

On finit par évaluer le modèle pour déterminer sa performance :

In [19]:
metric = evaluate.load("accuracy")
model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.26}